Approximate Background Noise

1) Average TARGET = 1 (S+N)

2) Average Target = 0 (N)

3) Average Train Data: Combined Target(0 & 1), Average(Target0 + Target1)/2

4) Average Test Data, Target unknown

5) Comparing Test Target=unknown [A0,B,A1,C,A2,D]

6) Comparing Train Target=0 [A0,B,A1,C,A2,D]

7) Comparing Train Target=1 [A0,B,A1,C,A2,D]

8) Comparing Train Target=0 vs Target=1

9) Removing Background

9a) Load data

9b) Removing Background noise(target=0)

10) Analyzing Signals

10a) Narrowband

10b) Bright Pixel

10c) Synchronized Needle

10d) Rocket

11) Flattening the Spectrum (amplitude equalizer)

11a) Step1, avg_df1_no_bckgnd = Averaging (df1-avg_df0)

11b) Step 2, psd_mask_df1 = 1 / avg_df1_no_bckgnd, this is just the reciprocal

11c) Delta btw A0-B, A1-C, A2-D

11e) Sum Polarization/Asymmetry Polarization


In [ ]:
import os
import time
import math
import warnings
import numpy as np
import pandas as pd
#import tensorflow as tf
import matplotlib.pyplot as plt

#print(tf.__version__)


In [ ]:
EPS = 0.00001   #-5dB

In [ ]:
MAIN_DIR = '../input/seti-breakthrough-listen'
TRAIN_DIR = '../input/seti-breakthrough-listen/train'
SUB_DIR = '../input/seti-breakthrough-listen/test'

In [ ]:
train_df = pd.read_csv(os.path.join(MAIN_DIR, 'train_labels.csv'))
test_df = pd.read_csv(os.path.join(MAIN_DIR, 'sample_submission.csv'))

In [ ]:
def return_filpath(name, folder=TRAIN_DIR):
    path = os.path.join(folder, name[0], f'{name}.npy')
    return path

train_df['image_path'] = train_df['id'].apply(lambda x: return_filpath(x))
test_df['image_path'] = test_df['id'].apply(lambda x: return_filpath(x, folder=SUB_DIR))

df_0     = train_df[train_df['target']==0]
df_1     = train_df[train_df['target']==1]

In [ ]:
len(df_0)

In [ ]:
avg_sample = 10001 #debug, should use full available datas

**Approximate Background Noise**

**1) Average TARGET = 1 (S+N)**

In [ ]:
avg_df1 =np.zeros((6,273,256), dtype = np.float32)  #max_32bit = 2**31-1
max_31bit = 2**30-1

for i,fn in enumerate(df_1.image_path[:avg_sample]):
    spec   = np.load(fn)                           #6x273x256
    
    for k in range( len(avg_df1) ):
        if i == 0:
            avg_df1[k,] = spec[k,]

        else:
            #intensity is in 16bits
            avg_df1[k,] = avg_df1[k,] + spec[k,]  #32bit accumulator=16bit+log2(train_df.shape[0])= 16+15.6 = 31.6bits

        if(i%2000 == 0):
            dat_min=np.min(avg_df1[k,])
            dat_max=np.max(avg_df1[k,])
            print(k, dat_min, dat_max)

            if (np.abs(dat_max) > max_31bit):
                print(k,'close to overflow')
    
avg_df1 = avg_df1/(i+1)

In [ ]:
np.min(np.min(avg_df1,axis=1), axis=1)

In [ ]:
np.max(np.max(avg_df1,axis=1), axis=1)

In [ ]:
pos_df1 =np.zeros((6,273,256), dtype = np.float32)

pos_df1[0,] = avg_df1[0,]- np.min(avg_df1[0,])
pos_df1[1,] = avg_df1[1,]- np.min(avg_df1[1,])
pos_df1[2,] = avg_df1[2,]- np.min(avg_df1[2,])
pos_df1[3,] = avg_df1[3,]- np.min(avg_df1[3,])
pos_df1[4,] = avg_df1[4,]- np.min(avg_df1[4,])
pos_df1[5,] = avg_df1[5,]- np.min(avg_df1[5,])

In [ ]:
np.min(np.min(pos_df1,axis=1), axis=1), np.max(np.max(pos_df1,axis=1), axis=1)

In [ ]:
def plot_avg_noise(data, avg=True):
    fig, ax = plt.subplots(nrows=1,ncols=3,figsize=(24, 5)) 
    ax[0].plot(np.log(np.sum(data, axis=0)+EPS))
    ax[1].imshow( np.log(data+EPS).T, aspect = 0.5*float(data.shape[1]) / data.shape[0])
    ax[2].plot(np.log(np.sum(data.T, axis=0)+EPS))

    ax[0].set_xlabel('freq (bins)')
    ax[0].set_ylabel('intensity (x10 dB)')
    ax[0].set_title('power spec across freq')
    
    ax[1].set_xlabel('time (segments) ')
    ax[1].set_ylabel('freq (bins)')
    if avg:
        ax[1].set_title('avg S+N spec (x10 dB)')
    else:
        ax[1].set_title('S+N spec (x10 dB)')
        
    ax[2].set_xlabel('time (segments) ')
    ax[2].set_ylabel('intensity (x10 dB)')
    ax[2].set_title('power spec across time')

In [ ]:
#plot_avg_noise(pos_df1[0,])  #A0
#plot_avg_noise(pos_df1[1,])  #B
plot_avg_noise(pos_df1[2,])  #A1
#plot_avg_noise(pos_df1[3,])  #C
#plot_avg_noise(pos_df1[4,])  #A2
#plot_avg_noise(pos_df1[5,])  #D

**2) Average Target = 0 (background noises only)**

In [ ]:
avg_df0 =np.zeros((6,273,256), dtype = np.float32)  #max_32bit = 2**31-1
max_31bit = 2**30-1

for i,fn in enumerate(df_0.image_path[:avg_sample]):
    spec   = np.load(fn)                           #6x273x256
    
    for k in range( len(avg_df0) ):
        if i == 0:
            avg_df0[k,] = spec[k,]

        else:
            #intensity is in 16bits
            avg_df0[k,] = avg_df0[k,] + spec[k,]  #32bit accumulator=16bit+log2(train_df.shape[0])= 16+15.6 = 31.6bits

        if(i%2000 == 0):
            dat_min=np.min(avg_df0[k,])
            dat_max=np.max(avg_df0[k,])
            print(k, dat_min, dat_max)

            if (np.abs(dat_max) > max_31bit):
                print(k,'close to overflow')
    
avg_df0 = avg_df0/(i+1)

In [ ]:
pos_df0 =np.zeros((6,273,256), dtype = np.float32)

pos_df0[0,] = avg_df0[0,]- np.min(avg_df0[0,])
pos_df0[1,] = avg_df0[1,]- np.min(avg_df0[1,])
pos_df0[2,] = avg_df0[2,]- np.min(avg_df0[2,])
pos_df0[3,] = avg_df0[3,]- np.min(avg_df0[3,])
pos_df0[4,] = avg_df0[4,]- np.min(avg_df0[4,])
pos_df0[5,] = avg_df0[5,]- np.min(avg_df0[5,])

In [ ]:
#plot_avg_noise(pos_df0[0,])  #A0
#plot_avg_noise(pos_df0[1,])  #B
plot_avg_noise(pos_df0[2,])   #A1
#plot_avg_noise(pos_df0[3,])  #C
#plot_avg_noise(pos_df0[4,])  #A2
#plot_avg_noise(pos_df0[5,])  #D


**3) Average Train Data:** Combined Target(0 & 1), Average(Target0 + Target1)/2

In [ ]:
pos_df_trn =np.zeros((6,273,256), dtype = np.float32)  
pos_df_trn = (pos_df0 + pos_df1)/2

In [ ]:
#plot_avg_noise(pos_df_trn[0,])  #A0
#plot_avg_noise(pos_df_trn[1,])  #B
plot_avg_noise(pos_df_trn[2,])  #A1
#plot_avg_noise(pos_df_trn[3,])  #C
#plot_avg_noise(pos_df_trn[4,])  #A2
#plot_avg_noise(pos_df_trn[5,])  #D


**4) Average Test Data**, Target unknown

In [ ]:
avg_df_test =np.zeros((6,273,256), dtype = np.float32)  #max_32bit = 2**31-1
max_31bit = 2**30-1

for i,fn in enumerate(test_df.image_path[:avg_sample]):
    spec   = np.load(fn)                           #6x273x256
    
    for k in range( len(avg_df_test) ):
        if i == 0:
            avg_df1[k,] = spec[k,]

        else:
            #intensity is in 16bits
            avg_df_test[k,] = avg_df_test[k,] + spec[k,]  #32bit accumulator=16bit+log2(train_df.shape[0])= 16+15.6 = 31.6bits

        if(i%2000 == 0):
            dat_min=np.min(avg_df_test[k,])
            dat_max=np.max(avg_df_test[k,])
            print(k, dat_min, dat_max)

            if (np.abs(dat_max) > max_31bit):
                print(k,'close to overflow')
    
avg_df_test = avg_df_test/(i+1)

In [ ]:
pos_df_tst =np.zeros((6,273,256), dtype = np.float32)

pos_df_tst[0,] = avg_df_test[0,]- np.min(avg_df_test[0,])
pos_df_tst[1,] = avg_df_test[1,]- np.min(avg_df_test[1,])
pos_df_tst[2,] = avg_df_test[2,]- np.min(avg_df_test[2,])
pos_df_tst[3,] = avg_df_test[3,]- np.min(avg_df_test[3,])
pos_df_tst[4,] = avg_df_test[4,]- np.min(avg_df_test[4,])
pos_df_tst[5,] = avg_df_test[5,]- np.min(avg_df_test[5,])

In [ ]:
plot_avg_noise(pos_df_tst[0,])  #A0

In [ ]:
plot_avg_noise(pos_df_tst[1,])  #B

In [ ]:
plot_avg_noise(pos_df_tst[2,])  #A1

In [ ]:
plot_avg_noise(pos_df_tst[3,])  #C

In [ ]:
plot_avg_noise(pos_df_tst[4,])  #A2

In [ ]:
plot_avg_noise(pos_df_tst[5,])  #D

**5) Comparing Test Target**=unknown [A0,B,A1,C,A2,D]

In [ ]:
#Comparing Test ['A0','B','A1','C','A2','D']
fig, ax = plt.subplots(nrows=1,ncols=2,figsize=(24, 5)) 
#across time
ax[0].plot(np.log(np.sum(pos_df_tst[0,].T, axis=0)+EPS))
ax[0].plot(np.log(np.sum(pos_df_tst[1,].T, axis=0)+EPS))
ax[0].plot(np.log(np.sum(pos_df_tst[2,].T, axis=0)+EPS))
ax[0].plot(np.log(np.sum(pos_df_tst[3,].T, axis=0)+EPS))
ax[0].plot(np.log(np.sum(pos_df_tst[4,].T, axis=0)+EPS))
ax[0].plot(np.log(np.sum(pos_df_tst[5,].T, axis=0)+EPS))
#across freq
ax[1].plot(np.log(np.sum(pos_df_tst[0,], axis=0)+EPS))
ax[1].plot(np.log(np.sum(pos_df_tst[1,], axis=0)+EPS))
ax[1].plot(np.log(np.sum(pos_df_tst[2,], axis=0)+EPS))
ax[1].plot(np.log(np.sum(pos_df_tst[3,], axis=0)+EPS))
ax[1].plot(np.log(np.sum(pos_df_tst[4,], axis=0)+EPS))
ax[1].plot(np.log(np.sum(pos_df_tst[5,], axis=0)+EPS))

ax[1].set_xlabel('freq (bins)')
ax[1].set_ylabel('intensity (x10 dB)')
ax[1].set_title('power spec across freq')
ax[1].legend(['A0','B','A1','C','A2','D'])

ax[0].set_xlabel('time (segments)')
ax[0].set_ylabel('intensity (x10 dB)')
ax[0].set_title('power spec across time')
ax[0].legend(['A0','B','A1','C','A2','D'])

**6) Comparing Train Target=0** [A0,B,A1,C,A2,D]

In [ ]:
#Target = 0
#across time
fig, ax = plt.subplots(nrows=1,ncols=2,figsize=(24, 5)) 
ax[0].plot(np.log(np.sum(pos_df0[0,].T, axis=0)+EPS))
ax[0].plot(np.log(np.sum(pos_df0[1,].T, axis=0)+EPS))
ax[0].plot(np.log(np.sum(pos_df0[2,].T, axis=0)+EPS))
ax[0].plot(np.log(np.sum(pos_df0[3,].T, axis=0)+EPS))
ax[0].plot(np.log(np.sum(pos_df0[4,].T, axis=0)+EPS))
ax[0].plot(np.log(np.sum(pos_df0[5,].T, axis=0)+EPS))
#across freq
ax[1].plot(np.log(np.sum(pos_df0[0,], axis=0)+EPS))
ax[1].plot(np.log(np.sum(pos_df0[1,], axis=0)+EPS))
ax[1].plot(np.log(np.sum(pos_df0[2,], axis=0)+EPS))
ax[1].plot(np.log(np.sum(pos_df0[3,], axis=0)+EPS))
ax[1].plot(np.log(np.sum(pos_df0[4,], axis=0)+EPS))
ax[1].plot(np.log(np.sum(pos_df0[5,], axis=0)+EPS))

ax[1].set_xlabel('freq (bins)')
ax[1].set_ylabel('intensity (x10 dB)')
ax[1].set_title('power spec across freq')
ax[1].legend(['A0','B','A1','C','A2','D'])

ax[0].set_xlabel('time (segments)')
ax[0].set_ylabel('intensity (x10 dB)')
ax[0].set_title('power spec across time')
ax[0].legend(['A0','B','A1','C','A2','D'])

**7) Comparing Train Target=1** [A0,B,A1,C,A2,D]

In [ ]:
#Target = 1
fig, ax = plt.subplots(nrows=1,ncols=2,figsize=(24, 5)) 
#across time
ax[0].plot(np.log(np.sum(pos_df1[0,].T, axis=0)+EPS))
ax[0].plot(np.log(np.sum(pos_df1[1,].T, axis=0)+EPS))
ax[0].plot(np.log(np.sum(pos_df1[2,].T, axis=0)+EPS))
ax[0].plot(np.log(np.sum(pos_df1[3,].T, axis=0)+EPS))
ax[0].plot(np.log(np.sum(pos_df1[4,].T, axis=0)+EPS))
ax[0].plot(np.log(np.sum(pos_df1[5,].T, axis=0)+EPS))
#across freq
ax[1].plot(np.log(np.sum(pos_df1[0,], axis=0)+EPS))
ax[1].plot(np.log(np.sum(pos_df1[1,], axis=0)+EPS))
ax[1].plot(np.log(np.sum(pos_df1[2,], axis=0)+EPS))
ax[1].plot(np.log(np.sum(pos_df1[3,], axis=0)+EPS))
ax[1].plot(np.log(np.sum(pos_df1[4,], axis=0)+EPS))
ax[1].plot(np.log(np.sum(pos_df1[5,], axis=0)+EPS))

ax[1].set_xlabel('freq (bins)')
ax[1].set_ylabel('intensity (x10 dB)')
ax[1].set_title('power spec across freq')
ax[1].legend(['A0','B','A1','C','A2','D'])

ax[0].set_xlabel('time (segments)')
ax[0].set_ylabel('intensity (x10 dB)')
ax[0].set_title('power spec across time')
ax[0].legend(['A0','B','A1','C','A2','D'])

**8) Comparing Train Target=0 vs Target=1**

In [ ]:
#Across time
#Train, Target 0 vs Target 1
fig, ax = plt.subplots(nrows=1,ncols=6,figsize=(24, 5)) 
#across time
#df1
ax[0].plot(np.log(np.sum(pos_df1[0,].T, axis=0)+EPS))
ax[1].plot(np.log(np.sum(pos_df1[1,].T, axis=0)+EPS))
ax[2].plot(np.log(np.sum(pos_df1[2,].T, axis=0)+EPS))
ax[3].plot(np.log(np.sum(pos_df1[3,].T, axis=0)+EPS))
ax[4].plot(np.log(np.sum(pos_df1[4,].T, axis=0)+EPS))
ax[5].plot(np.log(np.sum(pos_df1[5,].T, axis=0)+EPS))
#df0
ax[0].plot(np.log(np.sum(pos_df0[0,].T, axis=0)+EPS))
ax[1].plot(np.log(np.sum(pos_df0[1,].T, axis=0)+EPS))
ax[2].plot(np.log(np.sum(pos_df0[2,].T, axis=0)+EPS))
ax[3].plot(np.log(np.sum(pos_df0[3,].T, axis=0)+EPS))
ax[4].plot(np.log(np.sum(pos_df0[4,].T, axis=0)+EPS))
ax[5].plot(np.log(np.sum(pos_df0[5,].T, axis=0)+EPS))

ax[0].set_xlabel('time (segments)')
ax[0].set_ylabel('intensity (x10 dB)')
ax[0].set_title('power spec across time')
ax[0].legend(['A0-df1','A0-df0'])
ax[1].legend(['B-df1','B-df0'])
ax[2].legend(['A1-df1','A1-df0'])
ax[3].legend(['C-df1','C-df0'])
ax[4].legend(['A2-df1','A2-df0'])
ax[5].legend(['D-df1','D-df0'])

In [ ]:
#Across freq
#Train, Target 0 vs Target 1
fig, ax = plt.subplots(nrows=1,ncols=6,figsize=(24, 5)) 
#df1
ax[0].plot(np.log(np.sum(pos_df1[0,], axis=0)+EPS))
ax[1].plot(np.log(np.sum(pos_df1[1,], axis=0)+EPS))
ax[2].plot(np.log(np.sum(pos_df1[2,], axis=0)+EPS))
ax[3].plot(np.log(np.sum(pos_df1[3,], axis=0)+EPS))
ax[4].plot(np.log(np.sum(pos_df1[4,], axis=0)+EPS))
ax[5].plot(np.log(np.sum(pos_df1[5,], axis=0)+EPS))
#df0
ax[0].plot(np.log(np.sum(pos_df0[0,], axis=0)+EPS))
ax[1].plot(np.log(np.sum(pos_df0[1,], axis=0)+EPS))
ax[2].plot(np.log(np.sum(pos_df0[2,], axis=0)+EPS))
ax[3].plot(np.log(np.sum(pos_df0[3,], axis=0)+EPS))
ax[4].plot(np.log(np.sum(pos_df0[4,], axis=0)+EPS))
ax[5].plot(np.log(np.sum(pos_df0[5,], axis=0)+EPS))

ax[0].set_xlabel('freq (bins)')
ax[0].set_ylabel('intensity (x10 dB)')
ax[0].set_title('power spec across freq')
ax[0].legend(['A0-df1','A0-df0'])
ax[1].legend(['B-df1','B-df0'])
ax[2].legend(['A1-df1','A1-df0'])
ax[3].legend(['C-df1','C-df0'])
ax[4].legend(['A2-df1','A2-df0'])
ax[5].legend(['D-df1','D-df0'])

**9) Removing Background**

**9a) Load data**

In [ ]:
#Target = 1
#(t,f,i) = (time, freq, intensity)
arr = np.load('../input/seti-breakthrough-listen/train/5/54e340be921d.npy').astype(np.float32)  #narrowband,A0(f80,i48), dc spike(f90,i70)

#A2(f160) doppler overlapped with dc spike at f160 + bright pixels noises.  maybe multi signals bright pixel A1(f105)->A2(f110)
#arr = np.load('../input/seti-breakthrough-listen/train/0/099e8de5c54a.npy').astype(np.float32)  #not bright pixel, narrow band

#arr = np.load('../input/seti-breakthrough-listen/train/0/0689caff3aab.npy').astype(np.float32)  #blank, un-visible, A1 


#arr = np.load('../input/seti-breakthrough-listen/train/0/0f6dda0952ea.npy').astype(np.float32)  #narowband,A0,A1,A2(f125,i55dB-52dB)
#arr = np.load('../input/seti-breakthrough-listen/train/2/20b6bba77e00.npy').astype(np.float32)  #narrowband,A0,A1,A2(f125,i57dB-60dB)
#arr = np.load('../input/seti-breakthrough-listen/train/8/87cca65ea82e.npy').astype(np.float32)  #square pulse narrowband, A2(f160,i60dB)
#arr = np.load('../input/seti-breakthrough-listen/train/0/09cd074d5b1c.npy').astype(np.float32)  #squiggle square pulse narrowband A0(t170,f168,i67), A2(f180,i72)
#arr = np.load('../input/seti-breakthrough-listen/train/0/0cb54f44a200.npy').astype(np.float32)  #squiggle square pulse narrowband
#arr = np.load('../input/seti-breakthrough-listen/train/1/15af5fff3abb.npy').astype(np.float32)  #squiggle square pulse narrowband, A2(f40)
#arr = np.load('../input/seti-breakthrough-listen/train/0/0fe5c29a86b0.npy').astype(np.float32)  #square pulse narrowband
#arr = np.load('../input/seti-breakthrough-listen/train/1/118a3d42b310.npy').astype(np.float32)  #square pulse narrowband
#arr = np.load('../input/seti-breakthrough-listen/train/0/0b4b11cd4692.npy').astype(np.float32)  #square pulse narrowband


#https://www.kaggle.com/tomooinubushi/easter-egg-detection-with-template-matching
#all eggs are periodic, have no doppler shift and target are ones.  Got to think a little harder.
#almost all eggs are hiding at the low freq band edge and have some kind of hiding pattern.

egg_paths=['../input/seti-breakthrough-listen/train/2/2503d7f6e5c4.npy', #kaggle (t20,f20)
           '../input/seti-breakthrough-listen/train/8/805a7f4cac38.npy', #kaggle (t70,f50) 
           '../input/seti-breakthrough-listen/test/e/e05a5e667d06.npy',  #kaggle (t120,f80)
           '../input/seti-breakthrough-listen/test/1/1397c4ab0e5c.npy',  #alien (t10,f50)
           '../input/seti-breakthrough-listen/train/4/4f7bb8cf2d15.npy', #alien (t60,f80)
           '../input/seti-breakthrough-listen/test/1/1725ceec6de4.npy',  #alien (t110,f110)
           '../input/seti-breakthrough-listen/test/7/72bc12d576e2.npy',  #rocket (t10,f20)
           '../input/seti-breakthrough-listen/train/6/6c12bab0aeb4.npy', #rocket (t60,f50)
           '../input/seti-breakthrough-listen/test/1/1e6e43ddc15a.npy']  #rocket (t110,f80)
             

egg_ids=['2503d7f6e5c4', 
        '805a7f4cac38', 
        'e05a5e667d06', 
        '1397c4ab0e5c', 
        '4f7bb8cf2d15', 
        '1725ceec6de4', 
        '72bc12d576e2', 
        '6c12bab0aeb4', 
        '1e6e43ddc15a']


#arr = np.load(egg_paths[7]).astype(np.float32)  #

#fn='2503d7f6e5c4'
#fn='0689caff3aab'
#train_df[train_df['id']==fn].target

In [ ]:
arr_pos =np.zeros((6,273,256), dtype = np.float32) 

#remove negative value (linear scale)
arr_pos[0,] = arr[0,]- np.min(arr[0,])
arr_pos[1,] = arr[1,]- np.min(arr[1,])
arr_pos[2,] = arr[2,]- np.min(arr[2,])
arr_pos[3,] = arr[3,]- np.min(arr[3,])
arr_pos[4,] = arr[4,]- np.min(arr[4,])
arr_pos[5,] = arr[5,]- np.min(arr[5,])

In [ ]:
plot_avg_noise(arr_pos[0,], avg=False)  #A0

In [ ]:
plot_avg_noise(arr_pos[1,], avg=False)  #B

In [ ]:
plot_avg_noise( arr_pos[2,:,:], avg=False )  #A1

In [ ]:
plot_avg_noise(arr_pos[3,], avg=False)  #C

In [ ]:
plot_avg_noise(arr_pos[4,], avg=False)  #A2

In [ ]:
plot_avg_noise(arr_pos[5,], avg=False)  #D

**9b) Removing Background noise(target=0)**

In [ ]:
# Remove the noise
data_clean =np.zeros((6,273,256), dtype = np.float32) 

data_clean = arr_pos - pos_df0   #pos_df0 contains only N_bg  
data_clean[data_clean < 0] = 0.00001   #-5dB

In [ ]:
np.min(data_clean), np.min(arr_pos)

In [ ]:
fig, ax = plt.subplots(nrows=2,ncols=6,figsize=(24,4)) 

#original imag
ax[0,0].imshow(np.log(arr_pos[0,]+EPS).T, aspect = 0.5*float(arr_pos[0,].shape[1]) / arr_pos[0,].shape[0]) #, cmap="gray")
ax[0,1].imshow(np.log(arr_pos[1,]+EPS).T, aspect = 0.5*float(arr_pos[1,].shape[1]) / arr_pos[1,].shape[0]) #, cmap="gray")
ax[0,2].imshow(np.log(arr_pos[2,]+EPS).T, aspect = 0.5*float(arr_pos[2,].shape[1]) / arr_pos[2,].shape[0]) #, cmap="gray")
ax[0,3].imshow(np.log(arr_pos[3,]+EPS).T, aspect = 0.5*float(arr_pos[3,].shape[1]) / arr_pos[3,].shape[0]) #, cmap="gray")
ax[0,4].imshow(np.log(arr_pos[4,]+EPS).T, aspect = 0.5*float(arr_pos[4,].shape[1]) / arr_pos[4,].shape[0]) #, cmap="gray")
ax[0,5].imshow(np.log(arr_pos[5,]+EPS).T, aspect = 0.5*float(arr_pos[5,].shape[1]) / arr_pos[5,].shape[0]) #, cmap="gray")
#removed noise imag
ax[1,0].imshow(np.log(data_clean[0,]+EPS).T, aspect = 0.5*float(data_clean[0,].shape[1]) / data_clean[0,].shape[0]) #, cmap="gray")
ax[1,1].imshow(np.log(data_clean[1,]+EPS).T, aspect = 0.5*float(data_clean[1,].shape[1]) / data_clean[1,].shape[0]) #, cmap="gray")
ax[1,2].imshow(np.log(data_clean[2,]+EPS).T, aspect = 0.5*float(data_clean[2,].shape[1]) / data_clean[2,].shape[0]) #, cmap="gray")
ax[1,3].imshow(np.log(data_clean[3,]+EPS).T, aspect = 0.5*float(data_clean[3,].shape[1]) / data_clean[3,].shape[0]) #, cmap="gray")
ax[1,4].imshow(np.log(data_clean[4,]+EPS).T, aspect = 0.5*float(data_clean[4,].shape[1]) / data_clean[4,].shape[0]) #, cmap="gray")
ax[1,5].imshow(np.log(data_clean[5,]+EPS).T, aspect = 0.5*float(data_clean[5,].shape[1]) / data_clean[5,].shape[0]) #, cmap="gray")ax[1].set_xlabel('time')

ax[1,0].set_xlabel('time (segments)')
ax[1,0].set_ylabel('freq (bins))')
ax[1,0].set_title('spec A0')
ax[1,1].set_title('spec B')
ax[1,2].set_title('spec A1')
ax[1,3].set_title('spec C')
ax[1,4].set_title('spec A2')
ax[1,5].set_title('spec D')

In [ ]:
#Visualize across time

fig, ax = plt.subplots(nrows=1,ncols=6,figsize=(24, 5)) 
ax[0].plot(np.log(np.sum(arr_pos[0,].T, axis=0)+EPS),'b')    #S+N
ax[0].plot(np.log(np.sum(pos_df0[0,].T, axis=0)+EPS),'b--')  #avg target 0 as background noise
ax[0].plot(np.log(np.sum(pos_df_trn[0,].T, axis=0)+EPS),'g--')  #avg tran target= combined(0 & 1)
ax[0].plot(np.log(np.sum(pos_df_tst[0,].T, axis=0)+EPS),'c--')  #avg tran target= unknown
ax[0].plot(np.log(np.sum(data_clean[0,].T, axis=0)+EPS),'r')  #S+N - avg

ax[1].plot(np.log(np.sum(arr_pos[1,].T, axis=0)+EPS),'b')    #S+N
ax[1].plot(np.log(np.sum(pos_df0[1,].T, axis=0)+EPS),'b--')  #avg
ax[1].plot(np.log(np.sum(pos_df_trn[1,].T, axis=0)+EPS),'g--')  #avg tran target= combined(0 & 1)
ax[1].plot(np.log(np.sum(pos_df_tst[1,].T, axis=0)+EPS),'c--')  #avg tran target= unknown
ax[1].plot(np.log(np.sum(data_clean[1,].T, axis=0)+EPS),'r')  #S+N - avg

ax[2].plot(np.log(np.sum(arr_pos[2,].T, axis=0)+EPS),'b')    #S+N
ax[2].plot(np.log(np.sum(pos_df0[2,].T, axis=0)+EPS),'b--')  #avg
ax[2].plot(np.log(np.sum(pos_df_trn[2,].T, axis=0)+EPS),'g--')  #avg tran target= combined(0 & 1)
ax[2].plot(np.log(np.sum(pos_df_tst[2,].T, axis=0)+EPS),'c--')  #avg tran target= unknown
ax[2].plot(np.log(np.sum(data_clean[2,].T, axis=0)+EPS),'r')  #S+N - avg

ax[3].plot(np.log(np.sum(arr_pos[3,].T, axis=0)+EPS),'b')    #S+N
ax[3].plot(np.log(np.sum(pos_df0[3,].T, axis=0)+EPS),'b--')  #avg
ax[3].plot(np.log(np.sum(pos_df_trn[3,].T, axis=0)+EPS),'g--')  #avg tran target= combined(0 & 1)
ax[3].plot(np.log(np.sum(pos_df_tst[3,].T, axis=0)+EPS),'c--')  #avg tran target= unknown
ax[3].plot(np.log(np.sum(data_clean[3,].T, axis=0)+EPS),'r')  #S+N - avg

ax[4].plot(np.log(np.sum(arr_pos[4,].T, axis=0)+EPS),'b')    #S+N
ax[4].plot(np.log(np.sum(pos_df0[4,].T, axis=0)+EPS),'b--')  #avg
ax[4].plot(np.log(np.sum(pos_df_trn[4,].T, axis=0)+EPS),'g--')  #avg tran target= combined(0 & 1)
ax[4].plot(np.log(np.sum(pos_df_tst[4,].T, axis=0)+EPS),'c--')  #avg tran target= unknown
ax[4].plot(np.log(np.sum(data_clean[4,].T, axis=0)+EPS),'r')  #S+N - avg

ax[5].plot(np.log(np.sum(arr_pos[5,].T, axis=0)+EPS),'b')    #S+N
ax[5].plot(np.log(np.sum(pos_df0[5,].T, axis=0)+EPS),'b--')  #avg
ax[5].plot(np.log(np.sum(pos_df_trn[5,].T, axis=0)+EPS),'g--')  #avg tran target= combined(0 & 1)
ax[5].plot(np.log(np.sum(pos_df_tst[5,].T, axis=0)+EPS),'c--')  #avg tran target= unknown
ax[5].plot(np.log(np.sum(data_clean[5,].T, axis=0)+EPS),'r')  #S+N - avg

ax[0].set_xlabel('time (segments))')
ax[0].set_ylabel('intensity (x10 dB)')
ax[0].set_title('power spec across time')

ax[0].legend(['A0','avg_df0 A0','avg_trn A0','avg_tst A0','A0-avg_df0 A0'])
ax[1].legend(['B','avg_df0 B'])
ax[2].legend(['A1','avg_df0 A1'])
ax[3].legend(['C','avg_df0 C'])
ax[4].legend(['A2','avg_df0 A2'])
ax[5].legend(['D','avg_df0 D'])

In [ ]:
#Visualize across freq

fig, ax = plt.subplots(nrows=1,ncols=6,figsize=(24, 5)) 
ax[0].plot(np.log(np.sum(arr_pos[0,], axis=0)+EPS),'b')    #S+N
ax[0].plot(np.log(np.sum(pos_df0[0,], axis=0)+EPS),'b--')  #avg target 1
ax[0].plot(np.log(np.sum(pos_df_trn[0,], axis=0)+EPS),'g--')  #avg tran target= combined(0 & 1)
ax[0].plot(np.log(np.sum(pos_df_tst[0,], axis=0)+EPS),'c--')  #avg tran target= unknown
ax[0].plot(np.log(np.sum(data_clean[0,], axis=0)+EPS),'r')  #S+N - avg

ax[1].plot(np.log(np.sum(arr_pos[1,], axis=0)+EPS),'b')    #S+N
ax[1].plot(np.log(np.sum(pos_df0[1,], axis=0)+EPS),'b--')  #avg
ax[1].plot(np.log(np.sum(pos_df_trn[1,], axis=0)+EPS),'g--')  #avg tran target= combined(0 & 1)
ax[1].plot(np.log(np.sum(pos_df_tst[1,], axis=0)+EPS),'c--')  #avg tran target= unknown
ax[1].plot(np.log(np.sum(data_clean[1,], axis=0)+EPS),'r')  #S+N - avg

ax[2].plot(np.log(np.sum(arr_pos[2,], axis=0)+EPS),'b')    #S+N
ax[2].plot(np.log(np.sum(pos_df0[2,], axis=0)+EPS),'b--')  #avg
ax[2].plot(np.log(np.sum(pos_df_trn[2,], axis=0)+EPS),'g--')  #avg tran target= combined(0 & 1)
ax[2].plot(np.log(np.sum(pos_df_tst[2,], axis=0)+EPS),'c--')  #avg tran target= unknown
ax[2].plot(np.log(np.sum(data_clean[2,], axis=0)+EPS),'r')  #S+N - avg

ax[3].plot(np.log(np.sum(arr_pos[3,], axis=0)+EPS),'b')    #S+N
ax[3].plot(np.log(np.sum(pos_df0[3,], axis=0)+EPS),'b--')  #avg
ax[3].plot(np.log(np.sum(pos_df_trn[3,], axis=0)+EPS),'g--')  #avg tran target= combined(0 & 1)
ax[3].plot(np.log(np.sum(pos_df_tst[3,], axis=0)+EPS),'c--')  #avg tran target= unknown
ax[3].plot(np.log(np.sum(data_clean[3,], axis=0)+EPS),'r')  #S+N - avg

ax[4].plot(np.log(np.sum(arr_pos[4,], axis=0)+EPS),'b')    #S+N
ax[4].plot(np.log(np.sum(pos_df0[4,], axis=0)+EPS),'b--')  #avg
ax[4].plot(np.log(np.sum(pos_df_trn[4,], axis=0)+EPS),'g--')  #avg tran target= combined(0 & 1)
ax[4].plot(np.log(np.sum(pos_df_tst[4,], axis=0)+EPS),'c--')  #avg tran target= unknown
ax[4].plot(np.log(np.sum(data_clean[4,], axis=0)+EPS),'r')  #S+N - avg

ax[5].plot(np.log(np.sum(arr_pos[5,], axis=0)+EPS),'b')    #S+N
ax[5].plot(np.log(np.sum(pos_df0[5,], axis=0)+EPS),'b--')  #avg
ax[5].plot(np.log(np.sum(pos_df_trn[5,], axis=0)+EPS),'g--')  #avg tran target= combined(0 & 1)
ax[5].plot(np.log(np.sum(pos_df_tst[5,], axis=0)+EPS),'c--')  #avg tran target= unknown
ax[5].plot(np.log(np.sum(data_clean[5,], axis=0)+EPS),'r')  #S+N - avg

ax[0].set_xlabel('freq (bins))')
ax[0].set_ylabel('intensity (x10 dB)')
ax[0].set_title('power spec across freq')

ax[0].legend(['A0','avg_df0 A0','avg_trn A0','avg_tst A0','A0-avg_df0 A0'])
ax[1].legend(['B','avg_df0 B'])
ax[2].legend(['A1','avg_df0 A1'])
ax[3].legend(['C','avg_df0 C'])
ax[4].legend(['A2','avg_df0 A2'])
ax[5].legend(['D','avg_df0 D'])

**10) Analyzing Signals**

Plots are the same, only on the cleaned data.  A0_B, A1_C, A2_D overlap plots

In [ ]:
def plot_no_bckgnd(data_clean):    
    fig, ax = plt.subplots(nrows=2,ncols=3,figsize=(24, 10)) 
    #Visualize across time
    ax[0,0].plot(np.log(np.sum(data_clean[0,].T, axis=0)+EPS),'b')  #S+N - avg
    ax[0,0].plot(np.log(np.sum(data_clean[1,].T, axis=0)+EPS),'r--')  #S+N - avg
    ax[0,1].plot(np.log(np.sum(data_clean[2,].T, axis=0)+EPS),'b')  #S+N - avg
    ax[0,1].plot(np.log(np.sum(data_clean[3,].T, axis=0)+EPS),'g--')  #S+N - avg
    ax[0,2].plot(np.log(np.sum(data_clean[4,].T, axis=0)+EPS),'b')  #S+N - avg
    ax[0,2].plot(np.log(np.sum(data_clean[5,].T, axis=0)+EPS),'y--')  #S+N - avg
    ax[0,0].set_xlabel('time (segments))')
    ax[0,0].set_ylabel('intensity (x10 dB)')
    ax[0,0].set_title('power spec across time')
    ax[0,0].legend(['A0','B'])
    ax[0,1].legend(['A1','C'])
    ax[0,2].legend(['A2','D'])

    #Visualize across freq
    ax[1,0].plot(np.log(np.sum(data_clean[0,], axis=0)+EPS),'b')  #S+N - avg
    ax[1,0].plot(np.log(np.sum(data_clean[1,], axis=0)+EPS),'r--')  #S+N - avg
    ax[1,1].plot(np.log(np.sum(data_clean[2,], axis=0)+EPS),'b')  #S+N - avg
    ax[1,1].plot(np.log(np.sum(data_clean[3,], axis=0)+EPS),'g--')  #S+N - avg
    ax[1,2].plot(np.log(np.sum(data_clean[4,], axis=0)+EPS),'b')  #S+N - avg
    ax[1,2].plot(np.log(np.sum(data_clean[5,], axis=0)+EPS),'y--')  #S+N - avg
    ax[1,0].set_xlabel('freq (bins))')
    ax[1,0].set_ylabel('intensity (x10 dB)')
    ax[1,0].set_title('power spec across freq')
    ax[1,0].legend(['A0','B'])
    ax[1,1].legend(['A1','C'])
    ax[1,2].legend(['A2','D'])
    
    fig, ax = plt.subplots(nrows=1,ncols=6,figsize=(24,8)) 
    #removed noise imag
    ax[0].imshow(np.log(data_clean[0,]+EPS).T, aspect = 0.5*float(data_clean[0,].shape[1]) / data_clean[0,].shape[0]) 
    ax[1].imshow(np.log(data_clean[1,]+EPS).T, aspect = 0.5*float(data_clean[1,].shape[1]) / data_clean[1,].shape[0]) 
    ax[2].imshow(np.log(data_clean[2,]+EPS).T, aspect = 0.5*float(data_clean[2,].shape[1]) / data_clean[2,].shape[0])
    ax[3].imshow(np.log(data_clean[3,]+EPS).T, aspect = 0.5*float(data_clean[3,].shape[1]) / data_clean[3,].shape[0]) 
    ax[4].imshow(np.log(data_clean[4,]+EPS).T, aspect = 0.5*float(data_clean[4,].shape[1]) / data_clean[4,].shape[0]) 
    ax[5].imshow(np.log(data_clean[5,]+EPS).T, aspect = 0.5*float(data_clean[5,].shape[1]) / data_clean[5,].shape[0]) 

    ax[0].set_xlabel('time (segments)')
    ax[0].set_ylabel('freq (bins))')
    ax[0].set_title('spec A0')
    ax[1].set_title('spec B')
    ax[2].set_title('spec A1')
    ax[3].set_title('spec C')
    ax[4].set_title('spec A2')
    ax[5].set_title('spec D')

    return


def plot_clean_data(arr):
    
    arr_pos =np.zeros((6,273,256), dtype = np.float32) 
    #remove negative value (linear scale)
    arr_pos[0,] = arr[0,]- np.min(arr[0,])
    arr_pos[1,] = arr[1,]- np.min(arr[1,])
    arr_pos[2,] = arr[2,]- np.min(arr[2,])
    arr_pos[3,] = arr[3,]- np.min(arr[3,])
    arr_pos[4,] = arr[4,]- np.min(arr[4,])
    arr_pos[5,] = arr[5,]- np.min(arr[5,])

    # Remove the noise
    data_clean =np.zeros((6,273,256), dtype = np.float32) 
    data_clean = arr_pos - pos_df0   #pos_df0 contains only noises  
    data_clean[data_clean < 0] = 0.00001

    plot_no_bckgnd(data_clean)
    return

**10a) NarrowBand**

In [ ]:
arr = np.load('../input/seti-breakthrough-listen/train/5/54e340be921d.npy').astype(np.float32)  #narrowband,A0(f80,i48)
plot_clean_data(arr)  

Freq-domain :

A0 plot, Needle at tone 80, DC spike at tone 90.

A1 plot, suspect needle doppler shifted into DC spike.

A2 plot, probably doppler shifted into bursty RFI narrowband(f110-f200)

**10b) Bright-Pixel** ?

In [ ]:
#A2(f160) doppler(narrow band) and dc spike f160 + bright pixels noises, May be A1(f105)->A2(f110)
arr = np.load('../input/seti-breakthrough-listen/train/0/099e8de5c54a.npy').astype(np.float32)  #not bright pixel, need a larger plot to see a doppler shift at f160
plot_clean_data(arr) 

Freq-domain:

A2 plot: Bright pixel at tone 110, RFI at tones 90, 120, and doppler shift starting at tone 160(for sure).  DC spike at tone 160 as well.  Need a larger plot to see them.

A1 plot: DC spike, possible needle at tone 105 with intensity barely above the noise floor 67dB.  From A1 to A2, bright pixel tone 105 shfted to tone 110 (maybe)

C plot: DC spike tone 160. RFI tone 120.

A0 plot: noise floor is too high 65-67dB.

**10c) Synchronized Needle**

In [ ]:
# Needle is synchronized with RFI pulses
arr = np.load('../input/seti-breakthrough-listen/train/0/0689caff3aab.npy').astype(np.float32)  #blank, un-visible, A1 
plot_clean_data(arr) 

Time-domain: 

A0 plot, pulses are increasing in intensity, there are 2 signals pulses very close to each other(t46,t48),(t92,t96),. . more like synchronized.

A1 plot, pulses (t90,t92), t90 has the highest intensity.

Freq-domain:

very low noise floor.  The nominal training df1 noise floor is about 45dB. DC spike is about 85dB, this limited the dynamic range(16bit is about 96dB). 

The signal is under the mud, zero SNR. Need to knock down DC spike to bring up the signal.  Your NN is learning the DC spike, not needle.

**10d) Rocket**

In [ ]:
arr = np.load(egg_paths[7]).astype(np.float32)  #rocket
plot_clean_data(arr) 

Looks like AM modulation, high freq riding on very low freq wave.

**11) Flattening the Spectrum** (amplitude equalizer)

To flatten the receiver spectrum.  This is done by simple two steps.  The first step is to remove the background noise from the receiver signal, then averaged over many samples.

First Step: ( only show df_1, it can be df_trn, df_tst, df_0)

   avg_df1_no_bckgnd = avg( df_1 - avg_df0) <-- ~ avg(S) <-- avg( S+N - N )

Second Step:

   psd_mask_df1 = 1 / avg_df1_no_bckgnd,  this is just the reciprocal

Equalize the Spectrum:

data_clean = arr_pos - pos_df0   #  

data_clean_psd = data_clean * psd_mask_df1  # this is effectively amplitude equalizer

The advantage of this is on the band edges, it will equalize the amplitude and bias, such that the digital resolution is maintained.


**11a)  Step1, avg_df1_no_bckgnd** = Averaging (df1-avg_df0)

In [ ]:
#First Step

avg_df1_no_bckgnd =np.zeros((6,273,256), dtype = np.float32)  #max_32bit = 2**31-1
max_31bit = 2**30-1

for i,fn in enumerate(df_1.image_path[:avg_sample]):
    spec   = np.load(fn)                           #6x273x256
    
    for k in range( len(avg_df1_no_bckgnd) ):
        if i == 0:
            avg_df1_no_bckgnd[k,] = spec[k,]-avg_df0[k,]  #remove N_bg background noise

        else:
            #intensity is in 16bits
            avg_df1_no_bckgnd[k,] = avg_df1_no_bckgnd[k,] + spec[k,]-avg_df0[k,]  

        if(i%2000 == 0):
            dat_min=np.min(avg_df1_no_bckgnd[k,])
            dat_max=np.max(avg_df1_no_bckgnd[k,])
            print(k, dat_min, dat_max)

            if (np.abs(dat_max) > max_31bit):
                print(k,'close to overflow')
    
avg_df1_no_bckgnd = avg_df1_no_bckgnd/(i+1)

In [ ]:
df1_no_bckgnd_pos =np.zeros((6,273,256), dtype = np.float32)

df1_no_bckgnd_pos[0,] = avg_df1_no_bckgnd[0,]- np.min(avg_df1_no_bckgnd[0,])
df1_no_bckgnd_pos[1,] = avg_df1_no_bckgnd[1,]- np.min(avg_df1_no_bckgnd[1,])
df1_no_bckgnd_pos[2,] = avg_df1_no_bckgnd[2,]- np.min(avg_df1_no_bckgnd[2,])
df1_no_bckgnd_pos[3,] = avg_df1_no_bckgnd[3,]- np.min(avg_df1_no_bckgnd[3,])
df1_no_bckgnd_pos[4,] = avg_df1_no_bckgnd[4,]- np.min(avg_df1_no_bckgnd[4,])
df1_no_bckgnd_pos[5,] = avg_df1_no_bckgnd[5,]- np.min(avg_df1_no_bckgnd[5,])

df1_no_bckgnd_pos[df1_no_bckgnd_pos <= 0] = 1  #0dB

In [ ]:
#This is the spectrum shape of df1 with background noise removed.
plot_no_bckgnd(df1_no_bckgnd_pos)

In [ ]:
#this is natural log, limits 16bit and 17bit
np.log(2**15), np.log(2**-15),np.log(2**16), np.log(2**-16)

Freq-domain:

A0 plot, see the lower band-edge freq bin0-10 roll-off, and upper band-edge freq bin 220-256.
The plots are in natural log. The needle intensities on the band edges are weak.
The pass-band ripple is due to averaging over a small number of samples. 
In order to bring the band edges signal up is to have the amplitude equalizer. It is the mirror image of the spectrum. See Step 2.

**11b) Step 2, psd_mask_df1** = 1 / avg_df1_no_bckgnd,  this is just the reciprocal  

In [ ]:
#Second Step

ref_pwr = 1.0   #this can be used for data augmentation, ref_pwr +- deviation
psd_mask_df1 =np.zeros((6,273,256), dtype = np.float32)

for k in range(6):
    psd_mask_df1[k,] = ref_pwr / df1_no_bckgnd_pos[k,]  #linear scale   
    print('k=',k, np.min(df1_no_bckgnd_pos[k,]), '\t', np.max(df1_no_bckgnd_pos[k,]),'\t psd min= ', np.min(psd_mask_df1[k,]), 'max= ',np.max(psd_mask_df1[k,]))

In [ ]:
np.min(df1_no_bckgnd_pos), np.max(df1_no_bckgnd_pos), np.log(np.min(df1_no_bckgnd_pos)) , np.log(np.max(df1_no_bckgnd_pos)) 

In [ ]:
#This is reciprocal of df1_no_bckgnd_pos PSD
plot_no_bckgnd(psd_mask_df1)

Freq-domain:

A0 plot, notice the band-edges, they are reciprocal PSD of the above First Step.

In [ ]:
arr = np.load('../input/seti-breakthrough-listen/train/5/54e340be921d.npy').astype(np.float32)  #narrowband,A0(f80,i48)

arr_pos =np.zeros((6,273,256), dtype = np.float32) 
#remove negative value (linear scale)
arr_pos[0,] = arr[0,]- np.min(arr[0,])
arr_pos[1,] = arr[1,]- np.min(arr[1,])
arr_pos[2,] = arr[2,]- np.min(arr[2,])
arr_pos[3,] = arr[3,]- np.min(arr[3,])
arr_pos[4,] = arr[4,]- np.min(arr[4,])
arr_pos[5,] = arr[5,]- np.min(arr[5,])

# Remove the noise
data_clean =np.zeros((6,273,256), dtype = np.float32) 
data_clean = arr_pos - pos_df0   #S+N - N
data_clean[data_clean <= 0] = 1

#Equalize the Spectrum
data_clean_psd =np.zeros((6,273,256), dtype = np.float32)
data_clean_psd = data_clean*psd_mask_df1

plot_no_bckgnd(data_clean_psd)

**11c) Delta btw A0-B, A1-C, A2-D**

In [ ]:
#Delta btw A0-B, A1-C, A2-D
Delta =np.zeros((3,273,256), dtype = np.float32)
Delta[0] =data_clean_psd[0,] - data_clean_psd[1,]
Delta[1] =data_clean_psd[2,] - data_clean_psd[3,]
Delta[2] =data_clean_psd[4,] - data_clean_psd[5,]

Delta[Delta<=0] = 1

#plot Delta
plot_avg_noise(Delta[0,], False)  #Delta A0
plot_avg_noise(Delta[1,], False)  #Delta A1
plot_avg_noise(Delta[2,], False)  #Delta A2

**11e) Sum Polarization/Asymmetry Polarization**

In [ ]:
#https://github.com/ibm-watson-data-lab/ibmseti
#Sum polarization,       S = spect_L + spect_R
#Asymmetry polarization, A = (spect_L - spect_R) / (spect_L + spect_R)

#I'm not sure about the definition of polarization and this is done correctly.

#polarize,
pol =np.zeros((6,273,256), dtype = np.float32)

#sum_polarize, A0+B, A1+C, A2+D
pol[1,] = data_clean_psd[0,] + data_clean_psd[1,]
pol[3,] = data_clean_psd[2,] + data_clean_psd[3,]
pol[5,] = data_clean_psd[4,] + data_clean_psd[5,]

pol[pol ==0] = 1   #avoid divide by zero
#polarize, Asymmetry btw A0-B/(A0+B), (A1-C)/(A1+C), (A2-D)/(A2+D)
pol[0,] = (data_clean_psd[0,] - data_clean_psd[1,]) / pol[1]
pol[2,] = (data_clean_psd[2,] - data_clean_psd[3,]) / pol[3]
pol[4,] = (data_clean_psd[4,] - data_clean_psd[5,]) / pol[5]

pol[pol <=0 ] = 1

#plot polarization
plot_no_bckgnd(pol) 

In [ ]:
#Asymmetry polarization
plot_avg_noise(pol[0,], False)  #Asymmetry polarize A0
plot_avg_noise(pol[2,], False)  #Asymmetry polarize A2
plot_avg_noise(pol[4,], False)  #Asymmetry polarize A4


In [ ]:
#Sum polarization
plot_avg_noise(pol[1,], False)  #sum polarize A0+B
plot_avg_noise(pol[3,], False)  #sum polarize A1+C
plot_avg_noise(pol[5,], False)  #sum polarize A2+D

In [ ]:
def plot_all(arr):
    arr_pos =np.zeros((6,273,256), dtype = np.float32) 
    #remove negative value (linear scale)
    arr_pos[0,] = arr[0,]- np.min(arr[0,])
    arr_pos[1,] = arr[1,]- np.min(arr[1,])
    arr_pos[2,] = arr[2,]- np.min(arr[2,])
    arr_pos[3,] = arr[3,]- np.min(arr[3,])
    arr_pos[4,] = arr[4,]- np.min(arr[4,])
    arr_pos[5,] = arr[5,]- np.min(arr[5,])

    # Remove the noise
    data_clean =np.zeros((6,273,256), dtype = np.float32) 
    data_clean = arr_pos - pos_df0   #S+N -N
    data_clean[data_clean <= 0] = 1

    #Equalize the Spectrum
    data_clean_psd =np.zeros((6,273,256), dtype = np.float32)
    data_clean_psd = data_clean*psd_mask_df1

    plot_no_bckgnd(data_clean_psd)

    #Delta btw A0-B, A1-C, A2-D
    Delta =np.zeros((3,273,256), dtype = np.float32)
    Delta[0] =data_clean_psd[0,] - data_clean_psd[1,]
    Delta[1] =data_clean_psd[2,] - data_clean_psd[3,]
    Delta[2] =data_clean_psd[4,] - data_clean_psd[5,]

    Delta[Delta<=0] = 1

    #plot Delta
    plot_avg_noise(Delta[0,], False)  #Delta A0
    plot_avg_noise(Delta[1,], False)  #Delta A1
    plot_avg_noise(Delta[2,], False)  #Delta A2

    #polarize,
    pol =np.zeros((6,273,256), dtype = np.float32)

    #sum_polarize, A0+B, A1+C, A2+D
    pol[1,] = data_clean_psd[0,] + data_clean_psd[1,]
    pol[3,] = data_clean_psd[2,] + data_clean_psd[3,]
    pol[5,] = data_clean_psd[4,] + data_clean_psd[5,]

    pol[pol ==0] = 1   #avoid divide by zero
    #polarize, Asymmetry btw A0-B/(A0+B), (A1-C)/(A1+C), (A2-D)/(A2+D)
    pol[0,] = (data_clean_psd[0,] - data_clean_psd[1,]) / pol[1]
    pol[2,] = (data_clean_psd[2,] - data_clean_psd[3,]) / pol[3]
    pol[4,] = (data_clean_psd[4,] - data_clean_psd[5,]) / pol[5]

    pol[pol <=0 ] = 1

    #plot polarization
    plot_no_bckgnd(pol) 

    #Asymmetry polarization
    plot_avg_noise(pol[0,], False)  #Asymmetry polarize A0
    plot_avg_noise(pol[2,], False)  #Asymmetry polarize A2
    plot_avg_noise(pol[4,], False)  #Asymmetry polarize A4

    #Sum polarization
    plot_avg_noise(pol[1,], False)  #sum polarize A0+B
    plot_avg_noise(pol[3,], False)  #sum polarize A1+C
    plot_avg_noise(pol[5,], False)  #sum polarize A2+D

In [ ]:
arr = np.load('../input/seti-breakthrough-listen/train/0/099e8de5c54a.npy').astype(np.float32)  #not bright pixel, need a larger plot to see a doppler shift at f160
plot_all(arr)

In [ ]:
arr = np.load(egg_paths[7]).astype(np.float32)  #rocket,  A1(t180, f100-256)
plot_all(arr)

In [ ]:
arr = np.load(egg_paths[0]).astype(np.float32)  #kaggle
plot_all(arr)

For the model, i think the  "BiRNN multihead attention model", att_mh_rnn.py used in keywords spotting(kws)

https://github.com/google-research/google-research/tree/master/kws_streaming/models

is suitable for this application.  Target-On A, should pay attention to B,C,D.

